# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev1 toc-item"><a href="#Subgraph-of-Brain-Systems" data-toc-modified-id="Subgraph-of-Brain-Systems-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Subgraph of Brain Systems</a></div><div class="lev2 toc-item"><a href="#Convert-Lausanne-to-Brain-Systems-(84-ROI)" data-toc-modified-id="Convert-Lausanne-to-Brain-Systems-(84-ROI)-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Convert Lausanne to Brain Systems (84 ROI)</a></div><div class="lev2 toc-item"><a href="#Convert-Lausanne-to-Brain-Systems-(11-ROI)" data-toc-modified-id="Convert-Lausanne-to-Brain-Systems-(11-ROI)-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Convert Lausanne to Brain Systems (11 ROI)</a></div><div class="lev2 toc-item"><a href="#Filter-Subgraphs-with-Sparse-Expression" data-toc-modified-id="Filter-Subgraphs-with-Sparse-Expression-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Filter Subgraphs with Sparse Expression</a></div><div class="lev1 toc-item"><a href="#Task-Related-Constrasts" data-toc-modified-id="Task-Related-Constrasts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Task-Related Constrasts</a></div><div class="lev2 toc-item"><a href="#Between-Task-Subgraph-Contrast" data-toc-modified-id="Between-Task-Subgraph-Contrast-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Between-Task Subgraph Contrast</a></div><div class="lev2 toc-item"><a href="#Within-Task-Subgraph-Contrast" data-toc-modified-id="Within-Task-Subgraph-Contrast-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Within-Task Subgraph Contrast</a></div><div class="lev1 toc-item"><a href="#Task-Performance-and-Subgraph-Expression" data-toc-modified-id="Task-Performance-and-Subgraph-Expression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Task Performance and Subgraph Expression</a></div><div class="lev2 toc-item"><a href="#Load-Behavioral-Data" data-toc-modified-id="Load-Behavioral-Data-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Load Behavioral Data</a></div><div class="lev2 toc-item"><a href="#Reformulate-into-Data-Table" data-toc-modified-id="Reformulate-into-Data-Table-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Reformulate into Data Table</a></div><div class="lev2 toc-item"><a href="#Predicting-Interference-RT-(Hi)" data-toc-modified-id="Predicting-Interference-RT-(Hi)-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Predicting Interference RT (Hi)</a></div><div class="lev2 toc-item"><a href="#Predicting-Cost-RT-(Hi-Lo)" data-toc-modified-id="Predicting-Cost-RT-(Hi-Lo)-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Predicting Cost RT (Hi-Lo)</a></div><div class="lev2 toc-item"><a href="#Predicting-Interference-Accuracy-(Hi)" data-toc-modified-id="Predicting-Interference-Accuracy-(Hi)-45"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Predicting Interference Accuracy (Hi)</a></div><div class="lev2 toc-item"><a href="#Predicting-Cost-Accuracy-(Hi-Lo)" data-toc-modified-id="Predicting-Cost-Accuracy-(Hi-Lo)-46"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Predicting Cost Accuracy (Hi-Lo)</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division
from IPython.display import display

import os
import sys
import glob
import json
import subprocess

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

import fig_plotting
rcParams = fig_plotting.update_rcparams(rcParams)

import scipy.stats as stats

os.chdir('../')
import Codebase
conv_adj_matr_to_cfg_matr = Codebase.Networks.configuration.convert_adj_matr_to_cfg_matr
conv_cfg_vec_to_adj_matr = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr
os.chdir('./Analysis_Notebooks/')

path_Remotes = '/Users/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_RsrchData = path_Remotes + '/RSRCH.NMF_CogControl'
path_InpData = path_RsrchData + '/e02-FuncSubg'
path_ExpData = path_RsrchData + '/e04-FuncSubg_Dynamics'

for path in [path_CoreData, path_RsrchData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

# Subgraph of Brain Systems

In [ ]:
df_nmf = np.load('{}/NMF_Consensus.npz'.format(path_InpData), mmap_mode='r')
fac_subnet = df_nmf['fac_subnet']
fac_coef = df_nmf['fac_coef']
n_fac = fac_subnet.shape[0]
n_conn = fac_subnet.shape[1]
n_obs = fac_coef.shape[1]

path_cfg_expr = glob.glob('{}/NMF_Optimization.CfgMatr.npz'.format(path_InpData))[0]
df_cfg = np.load(path_cfg_expr, mmap_mode='r')
cfg_key = df_cfg['cfg_key']
key_type = np.unique(cfg_key)
n_key = len(key_type)
n_block = 6 
n_subj = n_obs / (n_key*n_block)
fac_coef_subj = np.zeros((n_fac, n_subj, n_key, n_block))
for key_ii, key in enumerate(np.unique(cfg_key)):
    key_ix = np.flatnonzero(cfg_key == key)
    fac_coef_subj[:, :, key_ii, :] = fac_coef[:, key_ix].reshape(n_fac, n_subj, n_block)
    
df_parcel = pd.read_csv('{}/LausanneScale125.csv'.format(path_CoreData))
lausanne_lbl = []
for lbl_id, lbl_roi, lbl_hemi in zip(df_parcel.Label_ID, df_parcel.ROI, df_parcel.Hemisphere):
    roi_name = '{}_{}'.format(lbl_hemi, lbl_roi)
    lausanne_lbl.append(roi_name)
lausanne_lbl = np.array(lausanne_lbl)

## Convert Lausanne to Brain Systems (84 ROI)

In [ ]:
df_parcel = pd.read_csv('{}/LausanneScale125.csv'.format(path_CoreData))

system_lbl = []
system_name = []
for lbl_id, lbl_roi, lbl_hemi in zip(df_parcel.Label_ID, df_parcel.ROI, df_parcel.Hemisphere):
    roi_name = '{}_{}'.format(lbl_hemi, lbl_roi.split('_')[0])
    
    if roi_name not in system_name:
        system_name.append(roi_name)
    system_lbl.append(roi_name)
system_lbl = np.array(system_lbl)
system_name = np.array(system_name)

n_system = len(system_name)
sys_triu_ix, sys_triu_iy = np.triu_indices(n_system, k=0)
n_sys_conn = len(sys_triu_ix)

n_perm = 1
alpha = 0.01

system_subgraph = []
for fac_i, subg in enumerate(fac_subnet):
    print('Processed: {} of {}'.format(fac_i+1, len(fac_subnet)))
    
    adj = conv_cfg_vec_to_adj_matr(subg)
    
    temp_system_subg = np.zeros(n_sys_conn)
    for tr_ii, (tr_ix, tr_iy) in enumerate(zip(sys_triu_ix, sys_triu_iy)):        
        ij_sys_ix = np.flatnonzero(system_lbl == system_name[tr_ix])
        ik_sys_ix = np.flatnonzero(system_lbl == system_name[tr_iy])  
        
        ij_ik_sys_ix, ik_ij_sys_ix = np.meshgrid(ij_sys_ix, ik_sys_ix)
        mean_conn = adj[ij_ik_sys_ix, ik_ij_sys_ix].mean()
        
        temp_system_subg[tr_ii] = mean_conn

        
    null_system_subg = np.zeros((n_perm, n_sys_conn))
    for perm_i in xrange(n_perm):
        for tr_ii, (tr_ix, tr_iy) in enumerate(zip(sys_triu_ix, sys_triu_iy)):
            system_lbl_perm = np.random.permutation(system_lbl)
            ij_sys_ix = np.flatnonzero(system_lbl_perm == system_name[tr_ix])
            ik_sys_ix = np.flatnonzero(system_lbl_perm == system_name[tr_iy])  

            ij_ik_sys_ix, ik_ij_sys_ix = np.meshgrid(ij_sys_ix, ik_sys_ix)
            mean_conn = adj[ij_ik_sys_ix, ik_ij_sys_ix].mean()

            null_system_subg[perm_i, tr_ii] = mean_conn
        
    pval_system_subg = np.mean(null_system_subg > temp_system_subg, axis=0)
    filt_system_subg = pval_system_subg.copy()
    filt_system_subg[pval_system_subg < alpha] = False
    filt_system_subg[pval_system_subg >= alpha] = True
    
    filt_ix = np.nonzero(filt_system_subg)
    thrsh_system_subg = temp_system_subg.copy()
    thrsh_system_subg[filt_ix] = 0
    
    temp_system_adj = np.zeros((n_system, n_system))
    temp_system_adj[sys_triu_ix, sys_triu_iy] = temp_system_subg
    temp_system_adj += np.triu(temp_system_adj, k=1).T

    thrsh_system_adj = np.zeros((n_system, n_system))
    thrsh_system_adj[sys_triu_ix, sys_triu_iy] = thrsh_system_subg
    thrsh_system_adj += np.triu(thrsh_system_adj, k=1).T
    
    
    # Format the system string output
    system_str = '\n'
    system_pair = []
    for sys_sig_ix in np.flatnonzero(filt_system_subg == 0):
        system_pair.append((system_name[sys_triu_ix[sys_sig_ix]],
                            system_name[sys_triu_iy[sys_sig_ix]]))
        system_str += '%20s <--> %-20s\n' % (system_pair[-1][0],
                                             system_pair[-1][1])
    
    # Generate subgraph dictionary
    system_subgraph.append({'Subgraph_ID': fac_i+1,
                            'full_system_subg': temp_system_adj,
                            'filt_system_subg': thrsh_system_adj,
                            'system_pairs': system_pair,
                            'system_string': system_str})

np.savez('{}/Subgraph.84System.npz'.format(path_ExpData),
         system_subgraph=system_subgraph,
         system_labels=system_lbl,
         system_names=system_name)

## Convert Lausanne to Brain Systems (11 ROI)

In [ ]:
df = h5py.File('{}/sysinfo234.mat'.format(path_CoreData), 'r')

ref = df['sysInfo']['region'][0, :]
region_lbl = [''.join(unichr(c) for c in df[rr])
              for rr in ref]
region_lbl = np.array(region_lbl)

ref = df['sysInfo']['system'][0, :]
system_lbl = [''.join(unichr(c) for c in df[rr])
              for rr in ref]
for ii in xrange(len(system_lbl), 262):
    system_lbl.append(u'cerebellum')
system_lbl = np.array(system_lbl)

system_name = np.unique(system_lbl)
n_system = len(system_name)
sys_triu_ix, sys_triu_iy = np.triu_indices(n_system, k=0)
n_sys_conn = len(sys_triu_ix)

n_perm = 1000
alpha = 0.01

system_subgraph = []
for fac_i, subg in enumerate(fac_subnet):
    print('Processed: {} of {}'.format(fac_i+1, len(fac_subnet)))
    
    adj = conv_cfg_vec_to_adj_matr(subg)
    
    temp_system_subg = np.zeros(n_sys_conn)
    for tr_ii, (tr_ix, tr_iy) in enumerate(zip(sys_triu_ix, sys_triu_iy)):        
        ij_sys_ix = np.flatnonzero(system_lbl == system_name[tr_ix])
        ik_sys_ix = np.flatnonzero(system_lbl == system_name[tr_iy])  
        
        ij_ik_sys_ix, ik_ij_sys_ix = np.meshgrid(ij_sys_ix, ik_sys_ix)
        mean_conn = adj[ij_ik_sys_ix, ik_ij_sys_ix].mean()
        
        temp_system_subg[tr_ii] = mean_conn

        
    null_system_subg = np.zeros((n_perm, n_sys_conn))
    for perm_i in xrange(n_perm):
        for tr_ii, (tr_ix, tr_iy) in enumerate(zip(sys_triu_ix, sys_triu_iy)):
            system_lbl_perm = np.random.permutation(system_lbl)
            ij_sys_ix = np.flatnonzero(system_lbl_perm == system_name[tr_ix])
            ik_sys_ix = np.flatnonzero(system_lbl_perm == system_name[tr_iy])  

            ij_ik_sys_ix, ik_ij_sys_ix = np.meshgrid(ij_sys_ix, ik_sys_ix)
            mean_conn = adj[ij_ik_sys_ix, ik_ij_sys_ix].mean()

            null_system_subg[perm_i, tr_ii] = mean_conn
        
    pval_system_subg = np.mean(null_system_subg > temp_system_subg, axis=0)
    filt_system_subg = pval_system_subg.copy()
    filt_system_subg[pval_system_subg < alpha] = False
    filt_system_subg[pval_system_subg >= alpha] = True
    
    filt_ix = np.nonzero(filt_system_subg)
    thrsh_system_subg = temp_system_subg.copy()
    thrsh_system_subg[filt_ix] = 0
    
    temp_system_adj = np.zeros((n_system, n_system))
    temp_system_adj[sys_triu_ix, sys_triu_iy] = temp_system_subg
    temp_system_adj += np.triu(temp_system_adj, k=1).T

    thrsh_system_adj = np.zeros((n_system, n_system))
    thrsh_system_adj[sys_triu_ix, sys_triu_iy] = thrsh_system_subg
    thrsh_system_adj += np.triu(thrsh_system_adj, k=1).T
    
    
    # Format the system string output
    system_str = '\n'
    system_pair = []
    for sys_sig_ix in np.flatnonzero(filt_system_subg == 0):
        system_pair.append((system_name[sys_triu_ix[sys_sig_ix]],
                            system_name[sys_triu_iy[sys_sig_ix]]))
        system_str += '%20s <--> %-20s\n' % (system_pair[-1][0],
                                             system_pair[-1][1])
    
    # Generate subgraph dictionary
    system_subgraph.append({'Subgraph_ID': fac_i+1,
                            'full_system_subg': temp_system_adj,
                            'filt_system_subg': thrsh_system_adj,
                            'system_pairs': system_pair,
                            'system_string': system_str})

np.savez('{}/Subgraph.11System.npz'.format(path_ExpData),
         system_subgraph=system_subgraph,
         system_labels=system_lbl,
         system_names=system_name)

## Filter Subgraphs with Sparse Expression

In [ ]:
# For every subgraph: compute the average expression sparsity over subjects
fac_coef_sparse = (fac_coef_subj == 0).reshape(n_fac, n_subj, -1).mean(axis=-1)
fac_coef_sparse_mean = fac_coef_sparse.mean(axis=-1)
fac_coef_sparse_std = fac_coef_sparse.std(axis=-1) / np.sqrt(n_subj)
ord_ix = np.argsort(fac_coef_sparse_mean)
fac_thresh = np.argmax(np.diff(fac_coef_sparse_mean[ord_ix]))
filter_fac_ix = ord_ix[:fac_thresh]

# Plot the distribution of temporal sparsity over subgraphs
% matplotlib inline
plt.figure()
ax = plt.subplot(111)
ax.plot(np.arange(n_fac), fac_coef_sparse_mean[ord_ix], 'k')
ax.fill_between(np.arange(n_fac),
                fac_coef_sparse_mean[ord_ix]-fac_coef_sparse_std[ord_ix],
                fac_coef_sparse_mean[ord_ix]+fac_coef_sparse_std[ord_ix])
ax.vlines(fac_thresh, 0, 1, 'r')

ax.set_xlim([-0.5, n_fac-0.5])        
plt.xticks(np.arange(0, n_fac+1, 6),
           np.arange(0, n_fac+1, 6))
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Ranked Subgraphs')

ax.set_ylim([0, 1])        
ax.set_yticks(np.linspace(0, 1.0, 5))
ax.yaxis.set_ticks_position('left')
ax.set_ylabel('Percent Sparse Coefficients')
plt.show()


# Load the systems level parcellation of subgraphs 
# Re-order subgraphs in terms of increasing expression sparsity
df_subg = np.load('{}/Subgraph.11System.npz'.format(path_ExpData))
sys_subgraph = df_subg['system_subgraph'][filter_fac_ix]
fac_coef_subj = fac_coef_subj[filter_fac_ix, ...]
n_subgraph = len(sys_subgraph)

# Plot the re-ordered systems-level subgraphs
% matplotlib inline
plt.figure(figsize=(12, 16))

vmin = np.min([np.min(ii['filt_system_subg']) for ii in sys_subgraph])
vmax = np.max([np.max(ii['filt_system_subg']) for ii in sys_subgraph])

for ii in xrange(len(sys_subgraph)):
    ax = plt.subplot(4, 5, ii+1)
    mat = ax.matshow(sys_subgraph[ii]['filt_system_subg'], cmap='viridis',
                     vmin=vmin, vmax=vmax)
    
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    ax.set_yticks(np.arange(len(df_subg['system_names'])))
    ax.set_xticks(np.arange(len(df_subg['system_names'])))
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.show()

# Task-Related Constrasts

## Between-Task Subgraph Contrast

In [ ]:
%matplotlib inline

grp_order = ['pos', 'neg']
tsk_order = ['Stroop', 'Navon']

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        subplot_dict = {'title': grp,
                        'xlabel': [],
                        'distrib': [],
                        'stats': []}
        
        for tsk in tsk_order:
            key_hi_ix = np.flatnonzero(key_type == 'adj_hi_{}_{}'.format(grp, tsk))
            key_lo_ix = np.flatnonzero(key_type == 'adj_lo_{}_{}'.format(grp, tsk))
            
            subj_hi = fac_coef_subj[fac_ii, :, key_hi_ix, :][0, ...]
            subj_lo = fac_coef_subj[fac_ii, :, key_lo_ix, :][0, ...]

            mean_subj_coef = 0.5*(np.mean(subj_hi, axis=1) + np.mean(subj_lo, axis=1))
            
            subplot_dict['xlabel'].append(tsk)
            subplot_dict['distrib'].append(mean_subj_coef)
        
        for key_pair in [[0,1]]:
            ts, pv = stats.ttest_rel(subplot_dict['distrib'][key_pair[0]],
                                     subplot_dict['distrib'][key_pair[1]]) 
            if np.isnan(pv):
                pv = 1.0
                            
            subplot_dict['stats'].append({'name': '{}-{}'.format(subplot_dict['xlabel'][key_pair[0]],
                                                                 subplot_dict['xlabel'][key_pair[1]]),
                                          'tstat': ts,
                                          'pval': pv,
                                          'sig': ''})
        grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
                sel_fac_between_sig.append(fac_ii)
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_between_sig = np.unique(sel_fac_between_sig)

# Plot each result
for fac_ix in sel_fac_between_sig: #xrange(n_subgraph): 
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])
    
    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.boxplot(gstat['distrib'])
        ax.set_xticklabels(gstat['xlabel'])
        ax.set_ylabel('Expression')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')


        for kstat in gstat['stats']:
            print('    %s-%s: ts: %0.6f pval: %0.6f sig: %s' % 
                  (gstat['title'], kstat['name'],
                   kstat['tstat'], kstat['pval'],
                   kstat['sig']))        
            
    plt.show()
    print('\n\n\n')

## Within-Task Subgraph Contrast

In [ ]:
%matplotlib inline

grp_order = ['pos_Stroop', 'pos_Navon', 'neg_Stroop', 'neg_Navon']
tsk_order = ['rs', 'lo', 'hi']

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        subplot_dict = {'title': grp,
                        'xlabel': [],
                        'distrib': [],
                        'stats': []}
        
        for tsk in tsk_order:
            key_ix = np.flatnonzero(key_type == 'adj_{}_{}'.format(tsk, grp))
            
            subj_coef = fac_coef_subj[fac_ii, :, key_ix, :][0, ...]

            mean_subj_coef = np.mean(subj_coef, axis=1)
            
            subplot_dict['xlabel'].append(tsk)
            subplot_dict['distrib'].append(mean_subj_coef)
            
        for key_pair in [[1,0], [2,0], [2,1]]:
            ts, pv = stats.ttest_rel(subplot_dict['distrib'][key_pair[0]],
                                     subplot_dict['distrib'][key_pair[1]])    
            if np.isnan(pv):
                pv = 1.0

            subplot_dict['stats'].append({'name': '{}-{}'.format(subplot_dict['xlabel'][key_pair[0]],
                                                                 subplot_dict['xlabel'][key_pair[1]]),
                                          'tstat': ts,
                                          'pval': pv,
                                          'sig': ''})        
        
        grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_within_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
                sel_fac_within_sig.append(fac_ii)
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_within_sig = np.unique(sel_fac_within_sig)

# Plot each result
for fac_ix in sel_fac_within_sig:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.boxplot(gstat['distrib'])
        ax.set_xticklabels(gstat['xlabel'])
        ax.set_ylabel('Expression')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')


        for kstat in gstat['stats']:
            print('    %s-%s: ts: %0.6f pval: %0.6f sig: %s' % 
                  (gstat['title'], kstat['name'],
                   kstat['tstat'], kstat['pval'],
                   kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')

# Task Performance and Subgraph Expression 

## Load Behavioral Data

In [ ]:
df_beh = io.loadmat('{}/BehavioralData_CogControlCorrect.mat'.format(path_CoreData))
df_blk = io.loadmat('{}/BlockwiseDataCorrectTrialsOnly.mat'.format(path_CoreData))

stroop_lbl = [lbl[0] for lbl in df_beh['StroopLabels'][0]]
navon_lbl = [lbl[0] for lbl in df_beh['NavonLabels'][0]]
stroop_beh = df_beh['StroopCorrectOnly']
navon_beh = df_beh['NavonCorrectOnly']

# Removed subjects
bad_subj_ix = [1, 6]
good_subj_ix = np.setdiff1d(np.arange(stroop_beh.shape[0]), bad_subj_ix)

# Excessive movement
stroop_move = io.loadmat('{}/StroopMove.mat'.format(path_CoreData))['move'][:, 0]
navon_move = io.loadmat('{}/NavonMove.mat'.format(path_CoreData))['move'][:, 0]

# Reaction Time
stroop_interf_rt = stroop_beh[good_subj_ix, 6]
navon_interf_rt = navon_beh[good_subj_ix, 6]

stroop_noninterf_rt = stroop_beh[good_subj_ix, 8]
navon_noninterf_rt = navon_beh[good_subj_ix, 8]

stroop_cost_rt = stroop_interf_rt - stroop_noninterf_rt
navon_cost_rt = navon_interf_rt - navon_noninterf_rt

# Accuracy
stroop_interf_acc = stroop_beh[good_subj_ix, 1]
navon_interf_acc = navon_beh[good_subj_ix, 1]

stroop_noninterf_acc = stroop_beh[good_subj_ix, 2]
navon_noninterf_acc = navon_beh[good_subj_ix, 2]

stroop_cost_acc = stroop_noninterf_acc - stroop_interf_acc
navon_cost_acc = navon_noninterf_acc - navon_interf_acc

print('Stroop Labels: {}'.format(stroop_lbl))
print('Navon Labels: {}'.format(navon_lbl))

"""
'high control accuracy', 'low control accuracy', 'high control mean RT',
'high control median RT', 'low control mean RT', 'low control median RT'
"""

## Reformulate into Data Table

In [ ]:
# Behavioral Data
df_blk = io.loadmat('{}/BlockwiseDataCorrectTrialsOnly.mat'.format(path_CoreData))
bad_subj_ix = [1, 6]
good_subj_ix = np.setdiff1d(np.arange(df_blk['StroopData'].shape[0]), bad_subj_ix)
n_subj = len(good_subj_ix)
n_perf_score = df_blk['StroopData'].shape[1]
n_block = df_blk['StroopData'].shape[2]

# Load Subgraph Factors as Systems
df_subg = np.load('{}/Subgraph.11System.npz'.format(path_ExpData))
sys_subgraph = df_subg['system_subgraph']
n_subgraph = len(sys_subgraph)

# Load Subgraph Factor Expression
path_cfg_expr = glob.glob('{}/NMF_Optimization.CfgMatr.npz'.format(path_InpData))[0]
df_cfg = np.load(path_cfg_expr, mmap_mode='r')
cfg_key = df_cfg['cfg_key']
key_grp = {'Stroop': {'Hi': {'Pos': 'adj_hi_pos_Stroop',
                             'Neg': 'adj_hi_neg_Stroop'},
                      'Lo': {'Pos': 'adj_lo_pos_Stroop',
                             'Neg': 'adj_lo_neg_Stroop'},
                      'Rs': {'Pos': 'adj_rs_pos_Stroop',
                             'Neg': 'adj_rs_neg_Stroop'}
                     },
           'Navon' : {'Hi': {'Pos': 'adj_hi_pos_Navon',
                             'Neg': 'adj_hi_neg_Navon'},
                      'Lo': {'Pos': 'adj_lo_pos_Navon',
                             'Neg': 'adj_lo_neg_Navon'},
                      'Rs': {'Pos': 'adj_rs_pos_Navon',
                             'Neg': 'adj_rs_neg_Navon'}
                     }
           }


# Populate data table
data_table = {'Subject_ID': [],
              'Subgraph_ID': [],
              'Task': [],
              'Epoch': [],
              'Block_ID': [],
              'Expr_Pos': [],
              'Expr_Neg': [],              
              'Mean_RT': [],
              'Median_RT': [],
              'Accuracy': []}
for fac_id in xrange(n_subgraph):                 
    for task_id in key_grp.keys():
        for epoch_id in key_grp[task_id].keys():
            key_pos_ix = np.flatnonzero(cfg_key == key_grp[task_id][epoch_id]['Pos'])
            key_neg_ix = np.flatnonzero(cfg_key == key_grp[task_id][epoch_id]['Neg'])

            fac_coef_subj_pos = fac_coef[fac_id, key_pos_ix].reshape(n_subj, n_block)
            fac_coef_subj_neg = fac_coef[fac_id, key_neg_ix].reshape(n_subj, n_block)
            
            for subj_ii, subj_id in enumerate(good_subj_ix):
                for block_id in xrange(n_block):

                    data_table['Subject_ID'].append(subj_id)
                    data_table['Subgraph_ID'].append(fac_id)
                    data_table['Task'].append(task_id)
                    data_table['Epoch'].append(epoch_id)
                    data_table['Block_ID'].append(block_id)
                    data_table['Expr_Pos'].append(fac_coef_subj_pos[subj_ii, block_id])
                    data_table['Expr_Neg'].append(fac_coef_subj_neg[subj_ii, block_id])

                    if epoch_id == 'Hi':
                        data_table['Mean_RT'].append(df_blk[task_id+'Data'][subj_id, 2, block_id])
                        data_table['Median_RT'].append(df_blk[task_id+'Data'][subj_id, 3, block_id])                        
                        data_table['Accuracy'].append(df_blk[task_id+'Data'][subj_id, 0, block_id])           

                    if epoch_id == 'Lo':
                        data_table['Mean_RT'].append(df_blk[task_id+'Data'][subj_id, 4, block_id])
                        data_table['Median_RT'].append(df_blk[task_id+'Data'][subj_id, 5, block_id])                        
                        data_table['Accuracy'].append(df_blk[task_id+'Data'][subj_id, 1, block_id])    

                    if epoch_id == 'Rs':
                        data_table['Mean_RT'].append(np.nan)
                        data_table['Median_RT'].append(np.nan)                        
                        data_table['Accuracy'].append(np.nan)                                                

# Save Data Table for R-stats
df = pd.DataFrame(data_table, columns=data_table.keys())
df.to_csv('./e04-R_Stats/subgraph_task_performance.csv')

## Predicting Interference RT (Hi)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['hi_pos', 'hi_neg']        
beh_grp = {'Stroop': [stroop_move, stroop_interf_rt],
           'Navon': [navon_move, navon_interf_rt]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_ix = np.flatnonzero(key_type == 'adj_{}_{}'.format(tsk, grp))
            
            subj_coef = fac_coef_subj[fac_ii, :, key_ix, :][0, ...]
            mean_subj_coef = np.mean(subj_coef, axis=-1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)

            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)

            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)


# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
for fac_ix in sel_fac_ord:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Blocks with Active Expression')
        ax.set_ylabel('Interference RT')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')    

## Predicting Cost RT (Hi-Lo)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['pos', 'neg']
beh_grp = {'Stroop': [stroop_move, stroop_cost_rt],
           'Navon': [navon_move, navon_cost_rt]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_hi_ix = np.flatnonzero(key_type == 'adj_hi_{}_{}'.format(tsk, grp))
            key_lo_ix = np.flatnonzero(key_type == 'adj_lo_{}_{}'.format(tsk, grp))
            
            subj_coef_hi = fac_coef_subj[fac_ii, :, key_hi_ix, :][0, ...]
            subj_coef_lo = fac_coef_subj[fac_ii, :, key_lo_ix, :][0, ...]
            mean_subj_coef = np.mean(subj_coef_hi, axis=1) - np.mean(subj_coef_lo, axis=1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)
            
            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)
                
            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})       
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
for fac_ix in sel_fac_ord[:10]:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Blocks with Active Expression (Hi-Lo)')
        ax.set_ylabel('Cost RT')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')    

## Predicting Interference Accuracy (Hi)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['hi_pos', 'hi_neg']        
beh_grp = {'Stroop': [stroop_move, stroop_interf_acc],
           'Navon': [navon_move, navon_interf_acc]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_ix = np.flatnonzero(key_type == 'adj_{}_{}'.format(tsk, grp))
            
            subj_coef = fac_coef_subj[fac_ii, :, key_ix, :][0, ...]
            mean_subj_coef = np.mean(subj_coef, axis=-1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)

            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)

            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)


# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
for fac_ix in sel_fac_ord:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Blocks with Active Expression')
        ax.set_ylabel('Interference Accuracy')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')    

## Predicting Cost Accuracy (Hi-Lo)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['pos', 'neg']
beh_grp = {'Stroop': [stroop_move, stroop_cost_acc],
           'Navon': [navon_move, navon_cost_acc]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_hi_ix = np.flatnonzero(key_type == 'adj_hi_{}_{}'.format(tsk, grp))
            key_lo_ix = np.flatnonzero(key_type == 'adj_lo_{}_{}'.format(tsk, grp))
            
            subj_coef_hi = fac_coef_subj[fac_ii, :, key_hi_ix, :][0, ...]
            subj_coef_lo = fac_coef_subj[fac_ii, :, key_lo_ix, :][0, ...]
            mean_subj_coef = np.mean(subj_coef_hi, axis=1) - np.mean(subj_coef_lo, axis=1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)
            
            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)
                
            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})       
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
for fac_ix in sel_fac_ord[:10]:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Blocks with Active Expression (Hi-Lo)')
        ax.set_ylabel('Cost Accuracy')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')    